In [1]:
import pandas as pd
import numpy as np
import datetime
from tqdm.notebook import tqdm
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
import requests
from bs4 import BeautifulSoup
import time
import re
from urllib.request import urlopen
from itertools import combinations, permutations
import matplotlib.pyplot as plt
import pickle

In [2]:
import preprocessing
import ModelEvaluator
import shutuba as sh

In [3]:
pd.set_option('display.max_columns', 50)

In [4]:
race_resluts = pd.read_pickle("../data/original_data/race_results.pickle")
horse_results = pd.read_pickle("../data/original_data/horse_results.pickle")
return_table = pd.read_pickle("../data/original_data/return_tables.pickle")

In [5]:
with open('le_horse.pickle', 'rb') as web:
  le_horse = pickle.load(web)
with open('le_jockey.pickle', 'rb') as web:
  le_jockey = pickle.load(web)

In [6]:
lgb_clf_danger = pickle.load(open('little_dangerous_pred.pkl', 'rb'))
lgb_clf_safety = pickle.load(open('very_safety.pkl', 'rb'))

In [7]:
p_horse_results = preprocessing.horse_preprocessing(horse_results)
m_race_results = pd.read_pickle("m_race_results.pickle")

c:\keiba\keiba\main\preprocessing.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["着順"] = pd.to_numeric(df["着順"], errors="coerce")
c:\keiba\keiba\main\preprocessing.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(subset=["着順"], inplace=True)
c:\keiba\keiba\main\preprocessing.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#return

In [25]:
horse_id_list = sh.shutuba_scraping_horse_id("202305010411")

In [26]:
set(horse_id_list) ^ set(p_horse_results[p_horse_results.index.isin(horse_id_list)].index.unique())

set()

In [10]:
popular =  [8,5,2,7,6,4,9,1,10,3]

In [11]:
shutuba = sh.shutuba_scrape(["202307011211"], "2023/02/05")
shutuba["人気"] = popular
p_shutuba = sh.shutuba_preprocessing(shutuba)
m_shutuba = preprocessing.merge_race_with_horse(p_shutuba, p_horse_results)

  0%|          | 0/1 [00:00<?, ?it/s]

In [13]:
d_shutuba = preprocessing.dummy_with_category(le_horse, le_jockey, m_shutuba, m_race_results)

In [14]:
d_shutuba

,枠番,馬番,斤量,人気,course_len,date,horse_id,jockey_id,age,weight,change_weight,n_horses,place,着順_3R,賞金_3R,着順/頭数_3R,着順_5R,賞金_5R,着順/頭数_5R,着順_allR,賞金_allR,着順/頭数_allR,着順_7R,賞金_7R,着順/頭数_7R,weather_曇,weather_晴,weather_小雨,weather_雨,weather_小雪,weather_雪,race_type_芝,race_type_ダート,race_type_障害,ground_state_良,ground_state_不良,ground_state_稍重,ground_state_重,gender_牡,gender_牝,gender_セ
202305010409,1,1,56,8,24.0,2023-02-05,25661,182,3,464,-6,10,5,7.000000,350.000000,60.317460,7.000000,350.000000,60.317460,7.000000,350.000000,60.317460,7.000000,350.000000,60.317460,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0
202305010409,2,2,56,5,24.0,2023-02-05,22843,55,3,512,-8,10,5,2.666667,416.666667,75.409544,3.250000,330.000000,72.628587,3.250000,330.000000,72.628587,3.250000,330.000000,72.628587,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0
202305010409,3,3,54,2,24.0,2023-02-05,24039,26,3,456,6,10,5,1.000000,700.000000,92.857143,1.000000,700.000000,92.857143,1.000000,700.000000,92.857143,1.000000,700.000000,92.857143,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0
202305010409,4,4,56,7,24.0,2023-02-05,24961,6,3,500,8,10,5,3.000000,401.500000,63.888889,3.000000,401.500000,63.888889,3.000000,401.500000,63.888889,3.000000,401.500000,63.888889,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0
202305010409,5,5,56,6,24.0,2023-02-05,23937,120,3,484,2,10,5,6.500000,350.000000,50.000000,6.500000,350.000000,50.000000,6.500000,350.000000,50.000000,6.500000,350.000000,50.000000,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0
202305010409,6,6,56,4,24.0,2023-02-05,24324,79,3,534,26,10,5,3.666667,233.333333,52.500000,3.666667,233.333333,52.500000,3.666667,233.333333,52.500000,3.666667,233.333333,52.500000,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0
202305010409,7,7,54,9,24.0,2023-02-05,24919,56,3,516,4,10,5,6.666667,134.266667,46.764347,5.250000,275.700000,55.906593,5.250000,275.700000,55.906593,5.250000,275.700000,55.906593,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0
202305010409,7,8,56,1,24.0,2023-02-05,24438,177,3,524,0,10,5,1.500000,400.000000,87.337662,1.500000,400.000000,87.337662,1.500000,400.000000,87.337662,1.500000,400.000000,87.337662,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0
202305010409,8,9,56,10,24.0,2023-02-05,25760,84,3,458,-8,10,5,5.000000,173.333333,36.666667,4.500000,175.000000,47.142857,4.500000,175.000000,47.142857,4.500000,175.000000,47.142857,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0
202305010409,8,10,54,3,24.0,2023-02-05,23130,126,3,424,-2,10,5,2.000000,482.850000,84.523810,2.000000,482.850000,84.523810,2.000000,482.850000,84.523810,2.000000,482.850000,84.523810,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0


In [15]:
d_shutuba_danger = d_shutuba.drop(columns=["date","人気"], axis=1)
d_shutuba_safety = d_shutuba.drop(columns="date", axis=1)

In [17]:
proba_danger = ModelEvaluator.predict_proba(lgb_clf_danger, d_shutuba_danger, train=False, minmax=False)
proba_safety = ModelEvaluator.predict_proba(lgb_clf_safety, d_shutuba_safety, train=False, minmax=False)

[LightGBM] [Warning] Unknown parameter: axis
[LightGBM] [Warning] Unknown parameter: axis


In [18]:
test = shutuba
test["danger_score"] = proba_danger
test["safety_score"] = proba_safety

In [19]:
test

,枠,馬番,印,馬名,性齢,斤量,騎手,厩舎,馬体重(増減),Unnamed:9_level_1,人気,登録,メモ,course_len,race_type,weather,ground_state,date,horse_id,jockey_id,danger_score,safety_score
202305010409,1,1,NaN,ボーンイングランデ,牡3,56.0,戸崎圭,栗東吉田,464(-6),---.-,8,NaN,NaN,2400,芝,晴,良,2023/02/05,2020106439,05386,-0.876949,-1.026732
202305010409,2,2,NaN,サヴォーナ,牡3,56.0,田辺,栗東中竹,512(-8),---.-,5,NaN,NaN,2400,芝,晴,良,2023/02/05,2020100734,01075,0.969782,0.086895
202305010409,3,3,NaN,エメリヨン,牝3,54.0,福永,美浦加藤士,456(+6),---.-,2,NaN,NaN,2400,芝,晴,良,2023/02/05,2020103120,01014,-0.555827,1.187058
202305010409,4,4,NaN,インスタキング,牡3,56.0,武豊,美浦蛯名正,500(+8),---.-,7,NaN,NaN,2400,芝,晴,良,2023/02/05,2020104843,00666,0.702367,-0.655085
202305010409,5,5,NaN,ノーブルクライ,牡3,56.0,坂井,栗東友道,484(+2),---.-,6,NaN,NaN,2400,芝,晴,良,2023/02/05,2020102976,01163,-1.280241,-0.628283
202305010409,6,6,NaN,ラスハンメル,牡3,56.0,藤岡康,栗東松永幹,534(+26),---.-,4,NaN,NaN,2400,芝,晴,良,2023/02/05,2020103554,01116,-0.467094,1.346761
202305010409,7,7,NaN,ウェイビー,牝3,54.0,石橋脩,栗東西村,516(+4),---.-,9,NaN,NaN,2400,芝,晴,良,2023/02/05,2020104764,01077,-0.332152,-1.016564
202305010409,7,8,NaN,スキルヴィング,牡3,56.0,ルメール,美浦木村,524(0),---.-,1,NaN,NaN,2400,芝,晴,良,2023/02/05,2020103722,05339,1.953255,1.334218
202305010409,8,9,NaN,フォーサイドナイン,牡3,56.0,三浦,栗東奥村豊,458(-8),---.-,10,NaN,NaN,2400,芝,晴,良,2023/02/05,2020106661,01122,-0.978280,-1.283831
202305010409,8,10,NaN,ミヤビ,牝3,54.0,武藤,美浦尾形,424(-2),---.-,3,NaN,NaN,2400,芝,晴,良,2023/02/05,2020101343,01169,0.865139,0.655564


In [22]:
test[["馬番","馬名","人気", "danger_score", "safety_score"]].sort_values("人気")

,馬番,馬名,人気,danger_score,safety_score
202305010409,8,スキルヴィング,1,1.953255,1.334218
202305010409,3,エメリヨン,2,-0.555827,1.187058
202305010409,10,ミヤビ,3,0.865139,0.655564
202305010409,6,ラスハンメル,4,-0.467094,1.346761
202305010409,2,サヴォーナ,5,0.969782,0.086895
202305010409,5,ノーブルクライ,6,-1.280241,-0.628283
202305010409,4,インスタキング,7,0.702367,-0.655085
202305010409,1,ボーンイングランデ,8,-0.876949,-1.026732
202305010409,7,ウェイビー,9,-0.332152,-1.016564
202305010409,9,フォーサイドナイン,10,-0.978280,-1.283831
